In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.cloud import language_v1
from datetime import datetime


def fetch_comments_with_sentiment_google_api(stock_code, start_page, end_page):
    #client = language_v1.LanguageServiceClient()
    all_comments = []

    for page_num in range(start_page, end_page + 1):
        url = f"http://guba.eastmoney.com/list,{stock_code},f_{page_num}.html"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            comments = soup.find_all('tr', class_='listitem')
            for comment in comments:
                read = comment.find('div', class_='read').text if comment.find('div', class_='read') else 'N/A'
                reply = comment.find('div', class_='reply').text if comment.find('div', class_='reply') else 'N/A'
                title_div = comment.find('div', class_='title')
                title = title_div.get_text(strip=True) if title_div else 'N/A'
                author = comment.find('div', class_='author').text if comment.find('div', class_='author') else 'N/A'
                update_time = comment.find('div', class_='update').text if comment.find('div', class_='update') else 'N/A'

                # 尝试获取详细的时间信息（假设存在于标签的某个属性中）
                # 例如: data-update-time="2023-04-01 14:30"
                detailed_time = comment.find('div', class_='update')['data-update-time'] if comment.find('div', class_='update') and 'data-update-time' in comment.find('div', class_='update').attrs else update_time

                #sentiment_score = get_sentiment_score_google_api(client, title)

                all_comments.append({
                    #"页码": page_num,
                    "Update time": detailed_time,  # 使用详细的时间
                    "Author": author,
                    "Texts": title,
                    "Reads": read,
                    "Comments": reply,
                    #"Sentiment": sentiment_score
                })
        else:
            print(f"请求失败，状态码：{response.status_code}")

    df = pd.DataFrame(all_comments)
    #df['Update time'] = pd.to_datetime(df['Update time'], format='%m-%d %H:%M')
    #df['Update time'] = df[Update time].strftime('%m-%d %H:%M')
    #df.drop(columns=['页码'], inplace=True)
    csv_file = f"comments_{stock_code}_p{start_page}_to_p{end_page}_score.csv"
    df.to_csv(csv_file, index=False, encoding='utf_8_sig')
    print(f"数据已保存到CSV文件：{csv_file}")

#def get_sentiment_score_google_api(client, text):
#    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
#    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment
#    sentiment_score = (sentiment.score + 1) * 5  # 调整情绪分数的范围
#    return round(sentiment_score, 1)

# 示例：爬取宁德时代股吧的第1页到第2页的评论，并保存到CSV，包含情绪得分
fetch_comments_with_sentiment_google_api('300750', 1, 8)


In [ ]:
df = pd.read_csv('comments_300750_p1_to_p8_score.csv')
df

In [ ]:
import pandas as pd

# 假设所有数据都是在2024年
# 首先，将'Update time'列的数据转换为字符串，前面加上年份
df['Update time'] = '2024-' + df['Update time']

# 将'Update time'列转换为datetime类型
df['Update time'] = pd.to_datetime(df['Update time'])


df_filtered['Update time'] = pd.to_datetime(df_filtered['Update time'])
# 添加一个新列'Date'，仅包含日期部分
df_filtered['Date'] = df_filtered['Update time'].dt.date

indices_to_increment = {}
indices_to_decrement_after_second = {}

# 对每个日期分组并找到需要增加12小时的索引
for name, group in df_filtered.groupby('Date'):
    found_first = False  # 用于标记是否找到了第一个符合条件的索引
    for i in group.index[:-1]:  # 忽略每个组的最后一个索引
        if df_filtered.at[i, 'Update time'] < df_filtered.at[i + 1, 'Update time']:
            if not found_first:
                indices_to_increment[name] = i  # 记录下每个日期的第一个需要增加12小时的索引
                found_first = True
            else:
                # 找到第二个符合条件的索引后，记录并跳出循环
                indices_to_decrement_after_second[name] = i + 1
                break

# 对需要增加时间的行及其之前的行进行增加12小时操作
for date, start_index in indices_to_increment.items():
    day_indices = df_filtered[df_filtered['Date'] == date].index
    indices_to_change = day_indices[day_indices <= start_index]
    df_filtered.loc[indices_to_change, 'Update time'] += pd.Timedelta(hours=12)

# 对找到的第二个符合条件的索引之后的行，如果小时数为12，则减去12小时
for date, start_index_after_second in indices_to_decrement_after_second.items():
    # 过滤出索引之后的所有行
    day_indices_after_second = df_filtered[(df_filtered['Date'] == date) & (df_filtered.index >= start_index_after_second)].index
    for idx in day_indices_after_second:
        if df_filtered.at[idx, 'Update time'].hour == 12:
            df_filtered.at[idx, 'Update time'] -= pd.Timedelta(hours=12)

# 定义你想要筛选的时间范围
start_time = '2024-04-02 00:00'
end_time = '2024-04-08 23:59'

# 使用条件筛选来选出这个时间范围内的行
df_filtered = df[(df['Update time'] >= start_time) & (df['Update time'] <= end_time)]

df_filtered['Update time'] = pd.to_datetime(df_filtered['Update time']) 
# 将'Update time'列格式化为"YYYY-MM-DD HH:MM"，这将去除秒 
df_filtered['Update time'] = df_filtered['Update time'].dt.strftime('%Y-%m-%d %H:%M')

# 保存筛选后的数据
csv_file = f"guba_crawl_ningde.csv"
df_filtered.to_csv(csv_file, index=False, encoding='utf_8_sig')
print(f"finished")

In [ ]:
df_filtered

In [25]:
import pandas as pd
'''
df_filtered['Update time'] = pd.to_datetime(df_filtered['Update time'])
# 添加一个新列'Date'，仅包含日期部分
df_filtered['Date'] = df_filtered['Update time'].dt.date

indices_to_increment = {}
indices_to_decrement_after_second = {}

# 对每个日期分组并找到需要增加12小时的索引
for name, group in df_filtered.groupby('Date'):
    found_first = False  # 用于标记是否找到了第一个符合条件的索引
    for i in group.index[:-1]:  # 忽略每个组的最后一个索引
        if df_filtered.at[i, 'Update time'] < df_filtered.at[i + 1, 'Update time']:
            if not found_first:
                indices_to_increment[name] = i  # 记录下每个日期的第一个需要增加12小时的索引
                found_first = True
            else:
                # 找到第二个符合条件的索引后，记录并跳出循环
                indices_to_decrement_after_second[name] = i + 1
                break

# 对需要增加时间的行及其之前的行进行增加12小时操作
for date, start_index in indices_to_increment.items():
    day_indices = df_filtered[df_filtered['Date'] == date].index
    indices_to_change = day_indices[day_indices <= start_index]
    df_filtered.loc[indices_to_change, 'Update time'] += pd.Timedelta(hours=12)

# 对找到的第二个符合条件的索引之后的行，如果小时数为12，则减去12小时
for date, start_index_after_second in indices_to_decrement_after_second.items():
    # 过滤出索引之后的所有行
    day_indices_after_second = df_filtered[(df_filtered['Date'] == date) & (df_filtered.index >= start_index_after_second)].index
    for idx in day_indices_after_second:
        if df_filtered.at[idx, 'Update time'].hour == 12:
            df_filtered.at[idx, 'Update time'] -= pd.Timedelta(hours=12)
'''
df_filtered['Update time'] = pd.to_datetime(df_filtered['Update time'])

# 添加一个新列'Date'，仅包含日期部分
df_filtered['Date'] = df_filtered['Update time'].dt.date

indices_to_increment = {}

# 对每个日期分组并找到需要增加12小时的索引
for name, group in df_filtered.groupby('Date'):
    for i in group.index[:-1]:  # 忽略每个组的最后一个索引，因为它没有下一行可以比较
        if df_filtered.at[i, 'Update time'] < df_filtered.at[i + 1, 'Update time']:
            indices_to_increment[name] = i  # 记录下每个日期的第一个需要增加12小时的索引
            break

# 遍历每个日期的分组，对需要增加时间的行及其之前的行进行增加12小时操作
for date, start_index in indices_to_increment.items():
    # 找到这一天的所有行
    day_indices = df_filtered[df_filtered['Date'] == date].index
    # 找到第一个需要增加时间的行及其之前的所有行
    indices_to_change = day_indices[day_indices <= start_index]
    # 增加12小时
    df_filtered.loc[indices_to_change, 'Update time'] += pd.Timedelta(hours=12)

'''
# 定义一个函数，用于在每个组内部找出Update time小于下一行的行
def find_rows_with_earlier_time(group):
    # 需要添加group = group.copy()以避免SettingWithCopyWarning警告
    group = group.copy()
    return group.loc[group['Update time'].shift(-1) > group['Update time']]

# 使用groupby按Date列分组，并应用上述函数
result = df_filtered.groupby('Date').apply(find_rows_with_earlier_time).reset_index(drop=True)
'''
'''
csv_file = "guba_crawl_ningde1.csv"
df_filtered.to_csv(csv_file, index=False, encoding='utf_8_sig')
'''

C:\Users\ASUS\AppData\Local\Temp\ipykernel_20488\1424026566.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Update time'] = pd.to_datetime(df_filtered['Update time'])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20488\1424026566.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Date'] = df_filtered['Update time'].dt.date


'\ncsv_file = "guba_crawl_ningde1.csv"\ndf_filtered.to_csv(csv_file, index=False, encoding=\'utf_8_sig\')\n'

In [26]:
df_filtered

,Update time,Author,Texts,Reads,Comments,Date
153,2024-04-08 23:57:00,GlrG,外面全都是固态电池了…请问你有什么想法么,479,9,2024-04-08
154,2024-04-08 23:50:00,投资三剑客,没有必要一直满仓，做波段保胜率会有更高收益！,610,4,2024-04-08
155,2024-04-08 23:30:00,拉萨第一深情,智己汽车的半固态电池是宁德做的吗？,161,4,2024-04-08
156,2024-04-08 22:38:00,基民11150G573l,今年是固态电池元年，清陶能源的半固态电池已经实现量产在智己L6上，实现1000+,2612,13,2024-04-08
157,2024-04-08 22:32:00,龙场悟dao,宁德时代与宇通推出长寿命商用车电池组,382,0,2024-04-08
...,...,...,...,...,...,...
635,2024-04-02 01:02:00,平常心去对待一切6940,$宁德时代(SZ300750)$,30,0,2024-04-02
636,2024-04-02 01:02:00,平常心去对待一切6940,$宁德时代(SZ300750)$,31,0,2024-04-02
637,2024-04-02 01:02:00,平常心去对待一切6940,$宁德时代(SZ300750)$,32,0,2024-04-02
638,2024-04-02 01:02:00,平常心去对待一切6940,$宁德时代(SZ300750)$,26,0,2024-04-02


In [149]:
df = pd.read_csv('guba_crawl_ningde1.csv')
df['Update time'] = pd.to_datetime(df['Update time']) 
df['Update time'] = df['Update time'].dt.strftime('%Y-%m-%d %H:%M')
df.drop('Date', axis=1, inplace=True)
csv_file = "guba_crawl_ningde1.csv"
df.to_csv(csv_file, index=False, encoding='utf_8_sig')

In [150]:
df

,Update time,Author,Texts,Reads,Comments
0,2024-04-04 23:34,中华小恐龙,YL过来谈的主要是新能源车，对于国内汽车出口不一定能有什么利好，毕竟是担忧丑国自,553,0
1,2024-04-04 23:17,股市征服者,沪指震荡回踩五日线，大湖股份，中润资源携手涨停，长安汽车，宁德时代何去何从？,1855,17
2,2024-04-04 23:05,富有大展de欧阳锋,固态电池还很遥远,1088,5
3,2024-04-04 23:00,东方盛,宁德时代晚上有实质上利好。,303,5
4,2024-04-04 22:24,华夏能源网,“分享电池技术”成宁德时代新战略：继福特后，又将与通用汽车合作建厂,261,1
...,...,...,...,...,...
355,2024-04-02 18:08,短线狙击小分队,4月2日早盘-中科曙光、耀皮玻璃、宁德时代、浪潮信息、光迅科技。,12892,47
356,2024-04-02 18:01,南山王老五,四月份第一金股，持股不动，调整补仓。,221,0
357,2024-04-02 17:42,尾盘决定一切,4.2早评：提前半月潜伏TCL科技终成正果，下车北方铜业、福蓉科技、万丰奥威。,9727,51
358,2024-04-02 17:23,龙头核心论,锂电最强小寡头，背靠宁德时代，利润3年大增350倍，500家机构扎堆！,1775,4


In [134]:
from google.cloud import language_v1
import os
import pandas as pd

# Set up authentication for Google Cloud Natural Language API
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "psychic-expanse-419117-0ced5445a62c.json"

# Instantiate a client
client = language_v1.LanguageServiceClient()

def analyze_sentiment(text):
    """
    Using the Google Cloud Natural Language API to analyze sentiment
    """
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment
    # Convert sentiment score from -1 (negative) to 1 (positive) scale to a 0-10 scale
    score = round((sentiment.score + 1) * 5, 1)
    return score

# Load your initial scraping result to a DataFrame
df = pd.read_csv('guba_crawl_biyadi1.csv')

# Ensure text column is string type
df['Texts'] = df['Texts'].astype(str)

# Apply sentiment analysis to each row in the 'text' column
df['Sentiment'] = df['Texts'].apply(analyze_sentiment)

# Save the DataFrame with the sentiment scores to a new CSV file
df.to_csv('guba_sentiments_ningde.csv', index=False, encoding='utf_8_sig')

print("Sentiment scoring completed and results saved to 'xueqiu_result_with_sentiments.csv'.")

Sentiment scoring completed and results saved to 'xueqiu_result_with_sentiments.csv'.
